# CNN vs Professor's MCTS7500 Dataset

Analyze the professor's `mcts7500_pool.pickle` and evaluate our trained CNN against it.

## 1. Load & Analyze Professor's Dataset

In [1]:
import os
import pickle
import numpy as np

# Paths: Colab (Drive) or local — Drive mount is optional
BASE = os.getcwd()
try:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE = '/content/drive/MyDrive'
    print('Using Google Drive')
except Exception as e:
    print('Drive mount skipped:', type(e).__name__)
    print('Using local/uploaded files. Upload mcts7500_pool.pickle to Colab if needed.')

candidates = [os.path.join(BASE, 'mcts7500_pool.pickle'), 'mcts7500_pool.pickle',
             os.path.join(os.getcwd(), 'mcts7500_pool.pickle'), '/content/mcts7500_pool.pickle']
PICKLE_PATH = next((p for p in candidates if os.path.exists(p)), candidates[0])
if not os.path.exists(PICKLE_PATH):
    raise FileNotFoundError('mcts7500_pool.pickle not found. Upload it to the project folder.')
print('Loading professor dataset from:', PICKLE_PATH)
with open(PICKLE_PATH, 'rb') as f:
    data = pickle.load(f)

print('Keys:', list(data.keys()))
print('README:', data['README'])
print()

board_x = np.array(data['board_x'], dtype=np.float32)
play_y = np.array(data['play_y'], dtype=np.int64)

print('Dataset structure:')
print(f'  board_x: {board_x.shape} (6x7, option-a encoding: +1/-1/0)')
print(f'  play_y:  {play_y.shape} (column 0-6)')
print(f'  Unique play values: {np.unique(play_y)}')
print(f'  Move distribution: {np.bincount(play_y, minlength=7)}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using Google Drive
Loading professor dataset from: /content/drive/MyDrive/mcts7500_pool.pickle


/tmp/ipython-input-9266901.py:23: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)


Keys: ['board_x', 'play_y', 'README']
README: this is with MCTS7500 - but it looks for wins and blocks before doing mcts..all boards are saved assuming that the next move is plus - this works by saving -1*board when it is minus turn...

Dataset structure:
  board_x: (265620, 6, 7) (6x7, option-a encoding: +1/-1/0)
  play_y:  (265620,) (column 0-6)
  Unique play values: [0 1 2 3 4 5 6]
  Move distribution: [35500 36530 39033 44842 38365 36213 35137]


## 2. Convert to 6x7x2 (Option B) for Our CNN

Professor's format: 6x7 with +1 (plus), -1 (minus), 0 (empty). All boards from plus's perspective.

Our CNN expects 6x7x2: ch0 = current player (plus), ch1 = opponent (minus).

In [2]:
def option_a_to_option_b(board):
    """Convert (6,7) +1/-1/0 to (6,7,2) for CNN. Current player = plus."""
    out = np.zeros((6, 7, 2), dtype=np.float32)
    out[:, :, 0] = (board == 1).astype(np.float32)   # plus/current
    out[:, :, 1] = (board == -1).astype(np.float32)  # minus/opponent
    return out

X_prof = np.array([option_a_to_option_b(b) for b in board_x], dtype=np.float32)
y_prof = play_y

print('Converted X_prof:', X_prof.shape)
print('Sample board (option b, ch0=plus ch1=minus):')
print(X_prof[0][:, :, 0], '\nch1:\n', X_prof[0][:, :, 1])

Converted X_prof: (265620, 6, 7, 2)
Sample board (option b, ch0=plus ch1=minus):
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0.]] 
ch1:
 [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 1.]]


## 3. Load Our CNN

In [3]:
try:
    import tensorflow as tf
    from tensorflow import keras
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'tensorflow'])
    import tensorflow as tf
    from tensorflow import keras

import os

# Model path: Drive or local/uploaded
drive_models = '/content/drive/MyDrive/Connect4_Combined/models'
local_models = os.path.join(os.getcwd(), 'Connect4_Combined', 'models')
MODEL_DIR = drive_models if os.path.exists(drive_models) else local_models

CNN_PATH = os.path.join(MODEL_DIR, 'connect4_cnn_best.keras')
if not os.path.exists(CNN_PATH):
    CNN_PATH = os.path.join(MODEL_DIR, 'connect4_cnn_final.keras')

print('Loading CNN from:', CNN_PATH)
cnn = keras.models.load_model(CNN_PATH)
cnn.summary()

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


Loading CNN from: /content/drive/MyDrive/Connect4_Combined/models/connect4_cnn_best.keras


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 150 variables whereas the saved optimizer has 154 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 6, 7, 2)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 6, 7, 128) │      2,432 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 6, 7, 128) │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 6, 7, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 6, 7, 128) │    147,584 │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 7, 128) │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 6, 7, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 6, 7, 128) │    147,584 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 7, 128) │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 6, 7, 128) │          0 │ batch_normalizat… │
│                     │                   │            │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 6, 7, 128) │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 6, 7, 128) │    147,584 │ re_lu_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 7, 128) │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 6, 7, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 6, 7, 128) │    147,584 │ re_lu_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 7, 128) │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 6, 7, 128) │          0 │ batch_normalizat… │
│                     │                   │            │ re_lu_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 6, 7, 128) │          0 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 6, 7, 128) │    147,584 │ re_lu_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 7, 128) │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 7,309,207 (27.88 MB)

 Trainable params: 2,434,951 (9.29 MB)

 Non-trainable params: 4,352 (17.00 KB)

 Optimizer params: 4,869,904 (18.58 MB)

## 4. Evaluate CNN on Professor's Dataset

In [4]:
BATCH = 2048
preds = cnn.predict(X_prof, batch_size=BATCH, verbose=1)
cnn_moves = np.argmax(preds, axis=1)

acc = np.mean(cnn_moves == y_prof)
top2_correct = sum(1 for i in range(len(y_prof)) 
                   if y_prof[i] in np.argsort(preds[i])[-2:])
top2_acc = top2_correct / len(y_prof)

print('='*50)
print('CNN vs Professor MCTS7500')
print('='*50)
print(f'Top-1 accuracy: {acc:.2%}')
print(f'Top-2 accuracy: {top2_acc:.2%}')
print(f'Samples: {len(y_prof):,}')

130/130 ━━━━━━━━━━━━━━━━━━━━ 65s 492ms/step
CNN vs Professor MCTS7500
Top-1 accuracy: 63.02%
Top-2 accuracy: 81.96%
Samples: 265,620


## 5. Per-Column Breakdown

In [5]:
print('Per-column: count | CNN correct | accuracy')
for col in range(7):
    mask = y_prof == col
    n = mask.sum()
    if n == 0:
        continue
    correct = (cnn_moves[mask] == y_prof[mask]).sum()
    print(f'  Col {col}: {n:>6,} | {correct:>6,} | {correct/n:.2%}')

Per-column: count | CNN correct | accuracy
  Col 0: 35,500 | 21,142 | 59.55%
  Col 1: 36,530 | 21,364 | 58.48%
  Col 2: 39,033 | 23,905 | 61.24%
  Col 3: 44,842 | 33,306 | 74.27%
  Col 4: 38,365 | 24,006 | 62.57%
  Col 5: 36,213 | 22,306 | 61.60%
  Col 6: 35,137 | 21,364 | 60.80%


## 6. Play CNN vs MCTS (Simulated Games)

Use professor's positions: at each position, CNN chooses a move. Compare to MCTS recommended move. Track "agreement rate" as we step through game-like sequences.

Simpler: sample N random positions and report head-to-head — when CNN and MCTS agree, count as CNN "winning" that decision. We already did that above (accuracy).

Alternative: run actual games — CNN plays, MCTS plays (we'd need MCTS code). Since we only have the dataset, we report accuracy on the full pool.

In [6]:
print('Summary: CNN matches professor MCTS on {:.1%} of positions.'.format(acc))
print('(Higher = CNN plays more like the professor\'s MCTS7500 bot.)')

Summary: CNN matches professor MCTS on 63.0% of positions.
(Higher = CNN plays more like the professor's MCTS7500 bot.)


## 7. Game-Play Evaluation: Policy vs Weak MCTS

Run actual games: Policy (CNN or Transformer with win/block + legal-move masking) vs Weak MCTS. Uses `policy_move_with_rules` so we never miss obvious wins or blocks.

In [ ]:
# Connect4 game engine + Weak MCTS + policy_move_with_rules (win/block + legal masking)
import random

class Connect4Eval:
    def __init__(self):
        self.board = np.zeros((6, 7), dtype=np.int8)
        self.heights = np.zeros(7, dtype=np.int8)
        self.current_player = 1
        self.winner = None
        self.move_count = 0

    def copy(self):
        g = Connect4Eval()
        g.board = self.board.copy()
        g.heights = self.heights.copy()
        g.current_player = self.current_player
        g.winner = self.winner
        g.move_count = self.move_count
        return g

    def legal_moves(self):
        return [c for c in range(7) if self.heights[c] < 6]

    def make_move(self, col):
        if self.heights[col] >= 6:
            return False
        row = self.heights[col]
        self.board[row, col] = self.current_player
        self.heights[col] += 1
        self.move_count += 1
        if self._check_win(row, col):
            self.winner = self.current_player
        elif self.move_count >= 42:
            self.winner = 0
        self.current_player *= -1
        return True

    def _check_win(self, row, col):
        player = self.board[row, col]
        if row <= 2 and np.sum(self.board[row:row+4, col]) == 4 * player:
            return True
        for c in range(max(0, col-3), min(4, col+1)):
            if np.sum(self.board[row, c:c+4]) == 4 * player:
                return True
        for dr, dc in [(1, 1), (1, -1)]:
            count = 1
            for sign in [1, -1]:
                r, c = row + sign*dr, col + sign*dc
                while 0 <= r < 6 and 0 <= c < 7 and self.board[r, c] == player:
                    count += 1
                    r += sign*dr
                    c += sign*dc
            if count >= 4:
                return True
        return False

    def is_terminal(self):
        return self.winner is not None

    def encode(self, perspective=1):
        b = self.board if perspective == 1 else -self.board
        enc = np.zeros((6, 7, 2), dtype=np.float32)
        enc[:, :, 0] = (b == 1).astype(np.float32)
        enc[:, :, 1] = (b == -1).astype(np.float32)
        return enc

def find_winning_move(game, player):
    """Return winning column for player, or None."""
    for col in game.legal_moves():
        test = game.copy()
        old_p = test.current_player
        test.current_player = player
        test.make_move(col)
        if test.winner == player:
            return col
        test.current_player = old_p
    return None

def policy_move_with_rules(game, model, perspective=1):
    """Win check -> block check -> model with legal-move masking."""
    win_col = find_winning_move(game, game.current_player)
    if win_col is not None:
        return win_col
    block_col = find_winning_move(game, -game.current_player)
    if block_col is not None:
        return block_col
    x = game.encode(perspective=perspective)[None, ...]
    probs = model.predict(x, verbose=0)[0]
    legal = game.legal_moves()
    mask = np.full(7, -1e9, dtype=np.float32)
    for c in legal:
        mask[c] = 0.0
    scores = probs + mask
    return int(np.argmax(scores))

class WeakMCTS:
    def __init__(self, sims=100):
        self.sims = sims

    def get_move(self, game):
        win = find_winning_move(game, game.current_player)
        if win is not None:
            return win
        block = find_winning_move(game, -game.current_player)
        if block is not None:
            return block
        return self._mcts(game, self.sims)

    def _mcts(self, game, sims):
        root_player = game.current_player
        stats = {}
        for _ in range(sims):
            node = game.copy()
            path = []
            while not node.is_terminal():
                state = hash(node.board.tobytes())
                path.append(state)
                if state not in stats:
                    stats[state] = [0, 0.0]
                moves = node.legal_moves()
                if not moves:
                    break
                best_move = None
                best_ucb = -1e9
                parent_visits = stats[state][0]
                for col in moves:
                    test = node.copy()
                    test.make_move(col)
                    child = hash(test.board.tobytes())
                    if child not in stats:
                        stats[child] = [0, 0.0]
                    visits, value = stats[child]
                    if visits == 0:
                        ucb = 1e9
                    else:
                        exploit = value / visits
                        explore = 1.4 * np.sqrt(np.log(parent_visits + 1) / visits)
                        ucb = exploit + explore
                    if ucb > best_ucb:
                        best_ucb = ucb
                        best_move = col
                node.make_move(best_move)
                if stats[hash(node.board.tobytes())][0] == 0:
                    path.append(hash(node.board.tobytes()))
                    break
            depth = 0
            while not node.is_terminal() and depth < 12:
                moves = node.legal_moves()
                if not moves:
                    break
                node.make_move(random.choice(moves))
                depth += 1
            result = 1.0 if node.winner == root_player else (-1.0 if node.winner == -root_player else 0.0)
            for st in path:
                if st in stats:
                    stats[st][0] += 1
                    stats[st][1] += result
        best_move, best_val = None, -1e9
        for col in game.legal_moves():
            test = game.copy()
            test.make_move(col)
            st = hash(test.board.tobytes())
            if st in stats and stats[st][0] > 0:
                val = stats[st][1] / stats[st][0]
            else:
                val = -1e9
            if val > best_val:
                best_val, best_move = val, col
        return best_move if best_move is not None else random.choice(game.legal_moves())

weak_mcts = WeakMCTS(sims=120)
print('Connect4Eval, WeakMCTS, policy_move_with_rules ready.')

In [ ]:
# Run games: Policy (CNN/Transformer) vs Weak MCTS
EVAL_GAMES = 50

def run_games(model, name):
    wins = losses = ties = 0
    for _ in range(EVAL_GAMES):
        game = Connect4Eval()
        while not game.is_terminal():
            if game.current_player == 1:
                col = policy_move_with_rules(game, model, perspective=1)
            else:
                col = weak_mcts.get_move(game)
            game.make_move(col)
        if game.winner == 1:
            wins += 1
        elif game.winner == -1:
            losses += 1
        else:
            ties += 1
    return wins, losses, ties

# CNN vs Weak MCTS
w, l, t = run_games(cnn, 'CNN')
print('='*50)
print('CNN (policy_move_with_rules) vs Weak MCTS (sims=120)')
print('='*50)
print(f'W: {w} | L: {l} | T: {t} | Games: {EVAL_GAMES}')
print(f'Win rate: {w/EVAL_GAMES:.1%}')

# Transformer vs Weak MCTS (load if available)
trans_path = os.path.join(MODEL_DIR, 'connect4_transformer_best.keras')
if not os.path.exists(trans_path):
    trans_path = os.path.join(MODEL_DIR, 'connect4_transformer_final.keras')
if os.path.exists(trans_path):
    print()
    transformer = keras.models.load_model(trans_path)
    w2, l2, t2 = run_games(transformer, 'Transformer')
    print('='*50)
    print('Transformer (policy_move_with_rules) vs Weak MCTS (sims=120)')
    print('='*50)
    print(f'W: {w2} | L: {l2} | T: {t2} | Games: {EVAL_GAMES}')
    print(f'Win rate: {w2/EVAL_GAMES:.1%}')
else:
    print('\n(Transformer model not found; skipping Transformer vs MCTS)')